In [17]:
import numpy as np
import pandas as pd
import geopandas as gpd
# webscraping
import json
import urllib
from bs4 import BeautifulSoup
from urllib.request import urlopen
import os

In [22]:
path = os.getcwd()
home_path = os.path.dirname(path)
gis_folder = f'{home_path}\\GIS'

In [24]:
gis_folder

'C:\\Users\\david\\Documents\\@@ Python\\Jaar 5\\Q3\\ENVM1502\\Git\\GIS'

In [2]:
website_us = "https://waterservices.usgs.gov/nwis/site/?format=rdb&sites="

In [3]:
end = "&siteOutput=expanded&siteStatus=all"

In [4]:
ids =  [1312000,
        1315500,
        1318500,
        1327750,
        1328770,
        1335754,
        1335755,
        1358000,
        1359139,
        1359165,
        135980207,
        1372043,
        1372058,
        1374019,
        1376269,
        1376307,
        1376515,
        1376520]


In [5]:
# coords_lst = []
df_lst = []
for i in ids:
    site = website_us + f'0{i}' + end
    page = urlopen(site)
    html_bytes = page.read()
    html = html_bytes.decode("utf-8")
    splitted = html[2297:].split("\n")
    header = splitted[0].split('\t')
    data = splitted[2].split('\t')
    df_lst.append(pd.DataFrame(data=[data],columns=header))
#     data = np.genfromtxt(BeautifulSoup(html[2297:]))
#     new_data = data[~np.isnan(data)]
#     coords = new_data[3:5]
#     if coords[0] > 45:
#         coords = new_data[4:6]
#     coords_lst.append(coords)
# coords_lst = np.array(coords_lst)
# np.savetxt("station_number.txt",coords_lst,delimiter=',',header='"lat","lon"')

df_stations = pd.concat(df_lst)

In [12]:
gdf_station = gpd.GeoDataFrame(data=df_stations, 
                 geometry=gpd.points_from_xy(df_stations.dec_long_va,df_stations.dec_lat_va,crs="EPSG:26918"))

In [16]:
gdf_station.to_file("GIS\Stations.gpk", driver="GPKG")